In [12]:

from datetime import datetime
import functools
import os

from IPython.display import HTML, clear_output

import gymnasium as gym
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

try:
  import brax
except ImportError:
  !pip install git+https://github.com/google/brax.git@main
  clear_output()
  import brax

import torch
from brax import envs
from brax.envs import env as brax_env
from brax.envs.to_torch import JaxToTorchWrapper
from brax import jumpy as jp
from brax.io import html
from brax.io import model

from brax.training.agents.ppo import train as ppo
from brax.training.agents.sac import train as sac

if 'COLAB_TPU_ADDR' in os.environ:
  from jax.tools import colab_tpu
  colab_tpu.setup_tpu()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
DEVICE = "cpu"

In [5]:
env = gym.make('FetchSlide-v2', max_episode_steps=10)

In [10]:
gym_env = JaxToTorchWrapper(env, device=DEVICE)

In [11]:
gym_env

<JaxToTorchWrapper<TimeLimit<OrderEnforcing<PassiveEnvChecker<MujocoFetchSlideEnv<FetchSlide-v2>>>>>>

In [13]:
obs = gym_env.reset()

In [14]:
obs

({'observation': array([ 9.96056349e-01,  7.48908387e-01,  4.12666572e-01,  9.04922569e-01,
          6.83324140e-01,  4.14022562e-01, -9.11337793e-02, -6.55842470e-02,
          1.35598971e-03, -2.03562423e-06,  1.47158977e-03, -5.32964269e-03,
          1.24548654e-04, -1.88202238e-02,  1.22698261e-03, -5.56736934e-05,
          6.43864204e-05,  1.86450993e-02, -4.64845340e-04, -4.83036131e-03,
         -1.26969656e-03,  5.03838696e-05,  7.97511346e-06,  4.66194862e-07,
          5.49241789e-05]),
  'achieved_goal': array([0.90492257, 0.68332414, 0.41402256]),
  'desired_goal': array([1.67190182, 0.88547116, 0.41401894])},
 {})

In [15]:
action = torch.rand(gym_env.action_space.shape, device=DEVICE) * 2 - 1

In [16]:
action

tensor([ 0.8037, -0.1575, -0.6071,  0.4965])

In [18]:
gym_env.step(action)

ValueError: too many values to unpack (expected 4)

In [17]:
obs, reward, done, info = gym_env.step(action)

ValueError: too many values to unpack (expected 4)